# Response 1

In [1]:
import numpy as np
import plotly.graph_objects as go
from scipy.signal import lfilter

# Function to generate noisy sinusoidal data
def generate_noisy_sinusoid(frequency, sampling_rate, duration, noise_std_dev=0.01):
    time = np.arange(0, duration, 1/sampling_rate)
    frequency_noisy = frequency + np.random.normal(0, noise_std_dev, size=len(time))
    signal = np.sin(2 * np.pi * frequency_noisy * time)
    signal += np.random.normal(0, noise_std_dev, size=len(time))
    return signal

# Function to estimate pitch using autocorrelation
def estimate_pitch(signal, sampling_rate):
    def autocorrelate(signal):
        result = np.convolve(signal, signal[::-1], mode='full')
        return result[len(result)//2:]

    autocorrelation = autocorrelate(signal)
    peaks = np.diff(np.sign(np.diff(autocorrelation)))
    peak_indices = np.where(peaks < 0)[0] + 1
    if len(peak_indices) > 1:
        peak_index = peak_indices[1]  # Ignore the first peak at index 0
        pitch = sampling_rate / peak_index
    else:
        pitch = 0
    return pitch

# Generate fake input data
np.random.seed(0)
sampling_rate = 50
duration = 600  # 10 minutes
walking_signal = generate_noisy_sinusoid(2, sampling_rate, duration)
running_signal = generate_noisy_sinusoid(4, sampling_rate, duration)

# Plot a signal of both classes showing the pitch estimate
window_size = 2 * sampling_rate  # 2-second window
walking_window = walking_signal[:window_size]
running_window = running_signal[:window_size]

walking_pitch = estimate_pitch(walking_window, sampling_rate)
running_pitch = estimate_pitch(running_window, sampling_rate)

fig = go.Figure(data=[
    go.Scatter(x=np.arange(len(walking_window))/sampling_rate, y=walking_window, name='Walking'),
    go.Scatter(x=np.arange(len(running_window))/sampling_rate, y=running_window, name='Running')
])
fig.update_layout(title=f'Walking Pitch: {walking_pitch:.2f} Hz, Running Pitch: {running_pitch:.2f} Hz',
                  xaxis_title='Time (s)', yaxis_title='Acceleration')
fig.show()

# Response 2

In [2]:
import numpy as np
import plotly.graph_objects as go
from scipy.signal import find_peaks

# Function to generate fake data
def generate_fake_data(frequency, sampling_rate, duration, noise_amplitude=0.1, noise_frequency=0.01):
    time = np.arange(0, duration, 1/sampling_rate)
    signal = np.sin(2 * np.pi * (frequency + noise_frequency * np.random.randn(len(time))) * time)
    signal *= 1 + noise_amplitude * np.random.randn(len(time))
    return signal

# Function to estimate pitch using autocorrelation method
def estimate_pitch(signal, sampling_rate):
    autocorrelation = np.correlate(signal, signal, mode='full')[len(signal)-1:]
    autocorrelation = autocorrelation / autocorrelation[0]
    peaks, _ = find_peaks(autocorrelation, height=0.5)
    if len(peaks) > 0:
        period = peaks[0]
        frequency = sampling_rate / period
        return frequency
    else:
        return 0

# Generate fake data
walking_signal = generate_fake_data(2, 50, 600)
running_signal = generate_fake_data(4, 50, 600)

# Split data into training and testing sets
walking_train, walking_test = walking_signal[:len(walking_signal)//2], walking_signal[len(walking_signal)//2:]
running_train, running_test = running_signal[:len(running_signal)//2], running_signal[len(running_signal)//2:]

# Estimate pitch for a 2-second window
window_size = 2 * 50  # 2 seconds * 50 Hz
walking_window = walking_signal[:window_size]
running_window = running_signal[:window_size]

walking_pitch = estimate_pitch(walking_window, 50)
running_pitch = estimate_pitch(running_window, 50)

# Plot signals with pitch estimate
fig = go.Figure(data=[
    go.Scatter(x=np.arange(len(walking_window))/50, y=walking_window, name='Walking'),
    go.Scatter(x=np.arange(len(running_window))/50, y=running_window, name='Running')
])

fig.update_layout(
    title='Signals with Pitch Estimate',
    xaxis_title='Time (s)',
    yaxis_title='Acceleration',
    annotations=[
        dict(x=0.5, y=1.1, text=f'Walking Pitch: {walking_pitch:.2f} Hz', showarrow=False),
        dict(x=0.5, y=1.05, text=f'Running Pitch: {running_pitch:.2f} Hz', showarrow=False)
    ]
)

fig.show()

In [16]:
import numpy as np
import tensorflow as tf
from sklearn.metrics import accuracy_score
from scipy.signal import find_peaks

# Function to generate fake data
def generate_fake_data(frequency, sampling_rate, duration, noise_amplitude=0.1, noise_frequency=0.0005):
    time = np.arange(0, duration, 1/sampling_rate)
    signal = np.sin(2 * np.pi * (frequency + noise_frequency * np.random.randn(len(time))) * time)
    signal *= 1 + noise_amplitude * np.random.randn(len(time))
    return signal

# Function to estimate pitch using autocorrelation method
def estimate_pitch(signal, sampling_rate):
    autocorrelation = np.correlate(signal, signal, mode='full')[len(signal)-1:]
    autocorrelation = autocorrelation / autocorrelation[0]
    peaks, _ = find_peaks(autocorrelation, height=0.5)
    if len(peaks) > 0:
        period = peaks[0]
        frequency = sampling_rate / period
        return frequency
    else:
        return 0

# Generate fake data
walking_signal = generate_fake_data(2, 50, 600)
running_signal = generate_fake_data(4, 50, 600)

# Split data into training and testing sets
walking_train, walking_test = walking_signal[:len(walking_signal)//2], walking_signal[len(walking_signal)//2:]
running_train, running_test = running_signal[:len(running_signal)//2], running_signal[len(running_signal)//2:]

# Create training and testing datasets
window_size = 2 * 50  # 2 seconds * 50 Hz
train_data = []
train_labels = []
for i in range(0, len(walking_train), window_size):
    window = walking_train[i:i+window_size]
    train_data.append(window)
    train_labels.append(0)  # Label for walking
for i in range(0, len(running_train), window_size):
    window = running_train[i:i+window_size]
    train_data.append(window)
    train_labels.append(1)  # Label for running

test_data = []
test_labels = []
for i in range(0, len(walking_test), window_size):
    window = walking_test[i:i+window_size]
    test_data.append(window)
    test_labels.append(0)  # Label for walking
for i in range(0, len(running_test), window_size):
    window = running_test[i:i+window_size]
    test_data.append(window)
    test_labels.append(1)  # Label for running

train_data = np.array(train_data)
train_labels = np.array(train_labels)
test_data = np.array(test_data)
test_labels = np.array(test_labels)

# Define SSL model
model = tf.keras.Sequential([
    tf.keras.layers.Dense(64, activation='relu', input_shape=(window_size,)),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(16, activation='relu'),
    tf.keras.layers.Dense(1)
])

# Compile model
model.compile(optimizer='adam', loss='mean_squared_error')

# Train model
model.fit(train_data, train_labels, epochs=10, batch_size=32, verbose=2)

# Evaluate model
predicted_labels = model.predict(test_data)
predicted_labels = np.where(predicted_labels > 0.5, 1, 0)
print('Accuracy:', accuracy_score(test_labels, predicted_labels))

# Evaluate classification capacity of pitch estimate
pitch_estimates = []
for window in test_data:
    pitch = estimate_pitch(window, 50)
    pitch_estimates.append(pitch)

pitch_estimates = np.array(pitch_estimates)
classification = np.where(pitch_estimates > 3, 1, 0)  # Threshold for classification
print('Accuracy of pitch estimate:', accuracy_score(test_labels, classification))

Epoch 1/10
10/10 - 1s - loss: 0.2496 - 1s/epoch - 139ms/step
Epoch 2/10
10/10 - 0s - loss: 0.0678 - 49ms/epoch - 5ms/step
Epoch 3/10
10/10 - 0s - loss: 0.0310 - 37ms/epoch - 4ms/step
Epoch 4/10
10/10 - 0s - loss: 0.0169 - 34ms/epoch - 3ms/step
Epoch 5/10
10/10 - 0s - loss: 0.0101 - 38ms/epoch - 4ms/step
Epoch 6/10
10/10 - 0s - loss: 0.0078 - 35ms/epoch - 3ms/step
Epoch 7/10
10/10 - 0s - loss: 0.0061 - 33ms/epoch - 3ms/step
Epoch 8/10
10/10 - 0s - loss: 0.0050 - 38ms/epoch - 4ms/step
Epoch 9/10
10/10 - 0s - loss: 0.0042 - 42ms/epoch - 4ms/step
Epoch 10/10
10/10 - 0s - loss: 0.0037 - 34ms/epoch - 3ms/step
10/10 [==============================] - 0s 4ms/step
Accuracy: 0.8833333333333333
Accuracy of pitch estimate: 0.5


In [12]:
import numpy as np
import tensorflow as tf
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from scipy.signal import find_peaks

# Function to generate fake data
def generate_fake_data(frequency, sampling_rate, duration, noise_amplitude=0.1, noise_frequency=0.0005):
    time = np.arange(0, duration, 1/sampling_rate)
    signal = np.sin(2 * np.pi * (frequency + noise_frequency * np.random.randn(len(time))) * time)
    signal *= 1 + noise_amplitude * np.random.randn(len(time))
    return signal

# Function to estimate pitch using autocorrelation method
def estimate_pitch(signal, sampling_rate):
    autocorrelation = np.correlate(signal, signal, mode='full')[len(signal)-1:]
    autocorrelation = autocorrelation / autocorrelation[0]
    peaks, _ = find_peaks(autocorrelation, height=0.5)
    if len(peaks) > 0:
        period = peaks[0]
        frequency = sampling_rate / period
        return frequency
    else:
        return 0

# Generate fake data
walking_signal = generate_fake_data(2, 50, 600)
running_signal = generate_fake_data(4, 50, 600)

# Split data into training and testing sets
walking_train, walking_test = walking_signal[:len(walking_signal)//2], walking_signal[len(walking_signal)//2:]
running_train, running_test = running_signal[:len(running_signal)//2], running_signal[len(running_signal)//2:]

# Create training and testing datasets
window_size = 2 * 50  # 2 seconds * 50 Hz
train_data = []
train_labels = []
test_data = []
test_labels = []

for i in range(len(walking_train) // window_size):
    train_data.append(walking_train[i*window_size:(i+1)*window_size])
    train_labels.append(0)

for i in range(len(running_train) // window_size):
    train_data.append(running_train[i*window_size:(i+1)*window_size])
    train_labels.append(1)

for i in range(len(walking_test) // window_size):
    test_data.append(walking_test[i*window_size:(i+1)*window_size])
    test_labels.append(0)

for i in range(len(running_test) // window_size):
    test_data.append(running_test[i*window_size:(i+1)*window_size])
    test_labels.append(1)

train_data = np.array(train_data)
train_labels = np.array(train_labels)
test_data = np.array(test_data)
test_labels = np.array(test_labels)

# Create SSL model
model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(128, activation='relu', input_shape=(window_size,)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(1)
])

model.compile(optimizer='adam', loss='mean_squared_error')

# Train SSL model
model.fit(train_data, train_labels, epochs=10, batch_size=32)

# Evaluate classification capacity of pitch estimate
pitch_estimates = model.predict(test_data)
predicted_labels = (pitch_estimates > 0.5).astype(int)
print('Accuracy:', accuracy_score(test_labels, predicted_labels))
print('Classification Report:')
print(classification_report(test_labels, predicted_labels))
print('Confusion Matrix:')
print(confusion_matrix(test_labels, predicted_labels))

Epoch 1/10
10/10 [==============================] - 2s 4ms/step - loss: 0.1546
Epoch 2/10
10/10 [==============================] - 0s 5ms/step - loss: 0.0271
Epoch 3/10
10/10 [==============================] - 0s 4ms/step - loss: 0.0119
Epoch 4/10
10/10 [==============================] - 0s 4ms/step - loss: 0.0065
Epoch 5/10
10/10 [==============================] - 0s 5ms/step - loss: 0.0046
Epoch 6/10
10/10 [==============================] - 0s 5ms/step - loss: 0.0029
Epoch 7/10
10/10 [==============================] - 0s 5ms/step - loss: 0.0021
Epoch 8/10
10/10 [==============================] - 0s 5ms/step - loss: 0.0015
Epoch 9/10
10/10 [==============================] - 0s 4ms/step - loss: 0.0012
Epoch 10/10
10/10 [==============================] - 0s 5ms/step
Accuracy: 0.89
Classification Report:
              precision    recall  f1-score   support

           0       0.90      0.87      0.89       150
           1       0.88      0.91      0.89       150

    accuracy          

In [15]:
px.line(walking_signal)

In [14]:
import plotly.express as px